In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

market_price = pd.read_csv('market.csv')
standard_price = pd.read_csv('standard.csv')
coc = pd.read_csv('cost-of-cultivation.csv')
coc.head(),standard_price.head(),market_price.head()

#list of considered crops
products=list(coc['crop'])  # ["groundnut", "paddy", "rice", "wheat", "barley", "jowar", "bajra", "maize", "ragi"]
market_edited=market_price[market_price['commodity'].isin([x.capitalize() for x in products ])]
market_edited.head()

#take mean of all varieties of one crop
grouped_market = market_edited.groupby(['state', 'commodity']).agg(['mean']).reset_index()
grouped_market.columns = ["".join(x) for x in grouped_market.columns]
grouped_market.head()

#convert price in quintal to price in kg
grouped_market['modal_pricemean'] = grouped_market['modal_pricemean'].apply(lambda x: float(x)/100)
standard_price['price'] = standard_price['price'].apply(lambda x: float(x)/100)
grouped_market.head(),standard_price.head()

#convert to dictionary of crop to profit or cost
std_prices = dict(zip(standard_price.standard, standard_price.price))
coc_dict = dict(zip(coc.crop, coc.cost))
std_prices,coc_dict


#state vs crop matrix
yield_mat = pd.read_csv('state-crop-yield.csv')
yield_mat.head()

#state and crop lists
states = yield_mat['state']
crops = list(yield_mat.columns)[1:]
yield_np_mat = np.matrix(yield_mat)
prediction_table = {'State':[],
                    'Crop':[],
                    'Profit':[]}
yield_np_mat.shape


for i in range(0,yield_np_mat.shape[0]-1):
    for j in range(1,yield_np_mat.shape[1]):
        if(yield_np_mat[i,j]!=0.0):
	    #if yielding in current state then calculate profit
            rs_per_kg = grouped_market[(grouped_market['state']==states[i]) & (grouped_market['commodity'] == crops[j-1])]['modal_pricemean']
            if(rs_per_kg.shape[0]==1):
		#use market price data
                diff = ((yield_np_mat[i,j]) * rs_per_kg.values[0]) - coc_dict[str(crops[j-1].lower())]
                val = diff if diff>0.0 else 1.0
            else:
		#use approximated standard price
                diff = ((yield_np_mat[i,j]) * std_prices[str(crops[j-1].lower())]) - coc_dict[str(crops[j-1].lower())]
                val = diff if diff>0.0 else 1.0
            
	    #append all results to a table
            prediction_table['State'].append(states[i])
            prediction_table['Crop'].append(crops[j-1])
            prediction_table['Profit'].append(val)
        else:
	    #states which cant produce the given crop
            #print("**", states[i],crops[j-1])
            prediction_table['State'].append(states[i])
            prediction_table['Crop'].append(crops[j-1])
            prediction_table['Profit'].append(-1)



'''
#
plt.figure(figsize=((10,8)))
data1={ 'crop':[],
       'profit':[]
}
statess=input('Enter State Name:-')
for s in range (len(prediction_table['State'])):
    if(prediction_table['State'][s]==statess):
        if(prediction_table['Profit'][s]>1):
          data1['crop'].append(prediction_table['Crop'][s])

          data1['profit'].append(prediction_table['Profit'][s])   

#plt.pie(data1['profit'], labels = data1['crop'])

#data1=pd.DataFrame(data1)
#data1[['profit','crop']].describe()
'''

"\n#\nplt.figure(figsize=((10,8)))\ndata1={ 'crop':[],\n       'profit':[]\n}\nstatess=input('Enter State Name:-')\nfor s in range (len(prediction_table['State'])):\n    if(prediction_table['State'][s]==statess):\n        if(prediction_table['Profit'][s]>1):\n          data1['crop'].append(prediction_table['Crop'][s])\n\n          data1['profit'].append(prediction_table['Profit'][s])   \n\n#plt.pie(data1['profit'], labels = data1['crop'])\n\n#data1=pd.DataFrame(data1)\n#data1[['profit','crop']].describe()\n"

In [ ]:
import pandas as pd

profit_data = pd.read_csv("spd.csv")
profit_data = pd.DataFrame(np.matrix(profit_data),
           columns=('state','crop','profit')
                 )

profit_data['state']=profit_data['state'].astype('category')
profit_data['crop'] = profit_data['crop'].astype('category')

profit_data['state_code'] = profit_data['state'].cat.codes
profit_data['crop_code'] = profit_data['crop'].cat.codes

mid = pd.unique(profit_data[['state', 'state_code']].values.ravel('K'))
state_encoding = pd.DataFrame({"state":mid[:35], "code":mid[35:]})

mid1 = pd.unique(profit_data[['crop', 'crop_code']].values.ravel('K'))
crop_encoding = pd.DataFrame({"crop":mid1[:17], "code":mid1[17:]})

state_encoding.to_csv("statecode.csv", index=False)
crop_encoding.to_csv("cropcode.csv", index = False)

dataset = pd.DataFrame({"state": profit_data["state_code"], "crop": profit_data["crop_code"], "profit": profit_data["profit"].apply(lambda x: int(x))})
columns = ["state", "crop", "profit"]
dataset[columns].to_csv("encoded_dataset.csv", index = False, header = False)

In [ ]:
#pip uninstall tensorflow

In [ ]:
#pip install tensorflow==1.13.2

In [ ]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
from sklearn import preprocessing
from tensorflow.contrib.learn.python.learn.estimators import estimator
from tensorflow.contrib.tensor_forest.client import random_forest
from tensorflow.examples.tutorials.mnist import input_data
import argparse
import tempfile
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


df = pd.read_csv("cropdata.csv")

le = preprocessing.LabelEncoder()
l1 = df["Soil"]
le.fit(l1)
newsoil = le.transform(l1)
df["Soil"]=newsoil

l2 = df["Month"]
le.fit(l2)
df["Month"]=le.transform(l2)

l3 = df["State"]
le.fit(l3)
df["State"]=le.transform(l3)

df=df.iloc[:,1:]
df = pd.DataFrame(data = df.iloc[1:,:].values)

print(df)
df.to_csv('datafile.csv', index=False)


flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('model_dir', '', 'Base directory for output models.')
flags.DEFINE_string('data_dir', '/tmp/data/', 'Directory for storing data')

flags.DEFINE_integer('train_steps', '1000', 'Number of training steps.')
flags.DEFINE_string('batch_size', '1000','Number of examples in a training batch.')
flags.DEFINE_integer('num_trees', '100', 'Number of trees in the forest.')
flags.DEFINE_integer('max_nodes', '1000', 'Max total nodes in a single tree.')


def build_estimator(model_dir):
  params = tf.contrib.tensor_forest.python.tensor_forest.ForestHParams(
      num_classes=15, num_features=3,
      num_trees=FLAGS.num_trees, max_nodes=FLAGS.max_nodes)
  return random_forest.TensorForestEstimator(params, model_dir=model_dir)


def train_and_eval():
  model_dir = tempfile.mkdtemp() if not FLAGS.model_dir else FLAGS.model_dir
  print('model directory = %s' % model_dir)

  estimator = build_estimator(model_dir)

  early_stopping_rounds = 100
  check_every_n_steps = 100
  monitor = random_forest.LossMonitor(early_stopping_rounds,
                                                  check_every_n_steps)



  feat = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)

  estimator.fit(x=feat, y=mnist.train.labels,batch_size=FLAGS.batch_size,monitors=[monitor])

  results = estimator.evaluate(x=mnist.test.images, y=mnist.test.labels,
                               batch_size=FLAGS.batch_size)
  for key in sorted(results):
    print('%s: %s' % (key, results[key]))


def main(_):
  train_and_eval()


if __name__ == '__main__':
  tf.app.run()

     0   1   2   3   4   5   6   7   8   9   10  11
0     0   4   1   1   0   0   0   0   0   0   0   0
1     0   4   2   1   0   0   0   0   0   0   0   0
2     0   4   4   1   0   0   0   0   0   0   0   0
3     0   4   6   1   0   0   0   0   0   0   0   0
4     0   4   5   1   0   0   0   0   0   0   0   0
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
789   2   9  10   0   0   0   0   0   0   0   0   1
790   2   9  13   0   0   0   0   0   0   0   0   1
791   2   9  14   0   0   0   0   0   0   0   0   1
792   2   9  17   0   0   0   0   0   0   0   0   1
793   2   9  16   0   0   0   0   0   0   0   0   1

[794 rows x 12 columns]
model directory = /tmp/tmprgc63rue
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f93335ac4d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evalua

AttributeError: ignored